In [ ]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Flatten, Conv1D, MaxPooling1D, Dropout
from tensorflow.keras.optimizers import Adam , RMSprop, SGD

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
###load file and data preprocessing###

df_origin=pd.read_csv('file (Analyzed data) location')
df_origin = df_origin[500:18000]

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_origin)
df_ss = pd.DataFrame(df_scaled, columns = ['time', 'Average slip velocity (m/s)', 'Average axial shortening (mm)', 'Average friction', 'Average temperature (˚C)','Rate of temperature (˚C/s)', 'Rate of axial shortening (mm/s)'])

df_ss

In [ ]:
df_ss_friction = df_ss[['Average slip velocity (m/s)', 'Average axial shortening (mm)', 'Average temperature (˚C)', 'Rate of temperature (˚C/s)', 'Rate of axial shortening (mm/s)', 'Average friction']]
df_ss_friction

In [ ]:
###data split###

data = df_ss_friction
dataset = data.values
training_data_len = math.ceil(len(dataset)*0.85)
training_data = dataset[0:training_data_len, :]

In [ ]:
#set time interval
time_interval =           ###ref = 50, 100, 150, ... , 500

#Training data setting
X_train = []
y_train = []

for i in range(time_interval, training_data.shape[0]):
    X_train.append(training_data[i-time_interval:i, :-1])
    y_train.append(training_data[i, -1])

X_train, y_train = np.array(X_train), np.array(y_train)
X_train.shape, y_train.shape

#Validation data setting
validation_data_len = training_data_len+math.ceil((len(dataset)-training_data_len)*0.5)
print(validation_data_len)
val_data = dataset[training_data_len - time_interval:validation_data_len, :]

X_val = []
y_val = []

for i in range(time_interval, val_data.shape[0]):
    X_val.append(val_data[i-time_interval:i, :-1])
    y_val.append(val_data[i, -1])

X_val, y_val = np.array(X_val), np.array(y_val)

#Test data setting
test_data = dataset[validation_data_len - time_interval:, :]

X_test = []
y_test = []

for i in range(time_interval, test_data.shape[0]):
    X_test.append(test_data[i-time_interval:i, :-1])
    y_test.append(test_data[i, -1])

X_test, y_test = np.array(X_test), np.array(y_test)

In [ ]:
###construct training model###

#Set hyperparameters
filters1 =       #ref = 512
filters2 =       #ref = 256
kernel_size1 =   #ref = 3
kernel_size2 =   #ref = 3
pool_size =      #ref = 1 to 2
unit1 =          #ref = 256 to 512
dropout1 =       #ref = 0.1 to 0.2
unit2 =          #ref = 256 to 512
dropout2 =       #ref = 0.1 to 0.2
unit3 =          #ref = 128 to 256
dropout3 =       #ref = 0.1 to 0.2
unit4 =          #ref = 64
dropout4 =       #ref = 0.1 to 0.2

#Built model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Flatten, Conv1D, MaxPooling1D, Dropout

# Input data shape (timesteps, features)
input_shape = (X_train.shape[1], X_train.shape[2])
output_units = 1

model = Sequential([
    Conv1D(filters=filters1, kernel_size=kernel_size1, activation='relu', input_shape=input_shape),
    Conv1D(filters=filters2, kernel_size=kernel_size2, activation='relu'),
    MaxPooling1D(pool_size=pool_size),

    GRU(units=unit1, return_sequences=True, activation='tanh', dropout=dropout1),
    GRU(units=unit2, return_sequences=True, activation='tanh', dropout=dropout2),
    GRU(units=unit3, return_sequences=True, activation='tanh', dropout=dropout3),
    GRU(units=unit4, return_sequences=False, activation='tanh', dropout=dropout4),

    Dense(units=output_units, activation='linear')
])

model.summary()

In [ ]:
#Setting training parameters
learning_rate =      #ref = 0.002
epochs =             #ref = 100
batch_size =         #ref = 128 to 256

#Training model
model.compile(loss = 'mse', optimizer = Adam(learning_rate = learning_rate))
callbacks = [keras.callbacks.ModelCheckpoint(filepath='{epoch:03d}.h5',monitor='val_loss')]
history = model.fit(X_train, y_train, epochs = epochs, validation_data = (X_val, y_val), batch_size=batch_size, callbacks=callbacks)

In [ ]:
#Load best model
model_best = keras.models.load_model('location of saved model')
#test model
y_pred_val = model_best.predict(X_val)
y_pred_test = model_best.predict(X_test)

#plot validation values
plt.figure(figsize = (20,10))
plt.xlabel('Time')
plt.ylabel('Value')
plt.plot(y_val, color = 'red', label = 'real')
plt.plot(y_pred_val, color = 'blue', label = 'pred|iction')
plt.legend()
plt.show()

#plot test values
plt.figure(figsize = (20,10))
plt.xlabel('Time')
plt.ylabel('Value')
plt.plot(y_test, color = 'red', label = 'real')
plt.plot(y_pred_test, color = 'blue', label = 'pred|iction')
plt.legend()
plt.show()

In [ ]:
###estimation of gradient importance###

#options
training_data_size = X_train.shape[0]
data_interval =    #ref = 10

#defining graindient_importance
def gradient_importance(seq, model):

    seq = tf.Variable(seq[np.newaxis,:,:], dtype=tf.float32)

    with tf.GradientTape() as tape:
        predictions = model(seq)

    grads = tape.gradient(predictions, seq)
    grads = tf.reduce_mean(grads, axis=1).numpy()[0]

    return grads


#calculating gradient importance
df_imp = pd.DataFrame(columns = ['Average slip velocity (m/s)', 'Average axial shortening (mm)', 'Average temperature (˚C)', 'Rate of temperature (˚C/s)', 'Rate of axial shortening (mm/s)'])

for i in range(1, training_data_size, data_interval):

    grad_imp = gradient_importance(X_train[i], model_best)
    df_imp = df_imp.append(pd.DataFrame([[grad_imp[0], grad_imp[1], grad_imp[2], grad_imp[3], grad_imp[4]]],
                                        columns = ['Average slip velocity (m/s)', 'Average axial shortening (mm)', 'Average temperature (˚C)', 'Rate of temperature (˚C/s)', 'Rate of axial shortening (mm/s)']))

re_index = list(range(1, training_data_size, data_interval))
df_imp.index = re_index
df_imp.to_csv("importance.csv")